In [ ]:
import json
from datetime import date, timedelta

from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType

In [ ]:
# Parse the JSON string
output_data = json.loads(bronze_output)

# Access individual variables
start_date = output_data.get("start_date")
silver_adls = output_data.get("silver_adls")
bronze_adls = output_data.get("bronze_adls")

# print(f"Start Date: {start_date}")
# print(f"Silver ADLS: {silver_adls}")
# print(f"Bronze ADLS: {bronze_adls}")

In [ ]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

In [ ]:
df

In [ ]:
df.head()

In [ ]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [ ]:
df

In [ ]:
df.head()

In [ ]:
# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

In [ ]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

In [ ]:
df.head()

In [ ]:
# Save the transformed DataFrame to the Silver container
silver_data = f"{silver_adls}earthquake_events_silver/"

# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_data)

In [ ]:
# Use mssparkutils.notebook.exit() to pass the JSON output to the pipeline
mssparkutils.notebook.exit(silver_data)